#  Example with Stanford SR830 Bundle
The BundleLockin is an instrument consisting of multiple Stanford Research Systems SR830
Lock-in Amplifiers.
The Instrument adds the functionallety of getting the trace
of the induvidial lockins.
The induvidial traces  share the setpoints  between lockins.

In [1]:
# IMPORTS

#%matplotlib notebook
import qcodes as qc
import os
import numpy as np
from SR830_ext import SR830_ext
from bundle_lockin import BundleLockin, do2d_multi
from qcodes.instrument.base import Instrument
from qcodes.utils.validators import Numbers, Arrays
from qcodes import initialise_or_create_database_at, load_or_create_experiment

tutorial_db_path = os.path.join(os.getcwd(), 'tutorial_doND.db')
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#1@C:\Users\Farzad\Documents\Rasmus\SR830ext\tutorial_doND.db
-----------------------------------------------------------------------------------
1-results-1-lockin_setpoints,lockin_trace-1
2-results-2-slow_v_now,lockin_setpoints,lockin_trace-20

## Dummy Generator 
Virtual instrument to be used instead of af DAC or other external setting parameter

In [2]:
class DummyGenerator(Instrument):

    def __init__(self, name, **kwargs):

        super().__init__(name, **kwargs)

        self.add_parameter('v_start',
                           initial_value=0,
                           unit='V',
                           label='v start',
                           vals=Numbers(0,1e3),
                           get_cmd=None,
                           set_cmd=None)

        self.add_parameter('v_stop',
                           initial_value=1,
                           unit='V',
                           label='v stop',
                           vals=Numbers(1,1e3),
                           get_cmd=None,
                           set_cmd=None)
        
        self.add_parameter('v_now',
                           initial_value=0,
                           unit='V',
                           label='v_now',
                           vals=Numbers(self.v_start(),self.v_stop()),
                           get_cmd=None,
                           set_cmd=None)

In [3]:
# The parameter to be set in the outer loop
slow = DummyGenerator('slow')

# The parameter to be set in the inner loop 
fast = DummyGenerator('fast')

### Connect to and Initialze the SR830s

In [4]:
sr = SR830_ext('lockin', 'GPIB0::1::INSTR')
#sr2 = SR830_ext('lockin2', 'GPIB0::2::INSTR')

sr.ch1_display('X')
sr.ch1_ratio('none')
#sr2.ch1_display('X')
#sr2.ch1_ratio('none')

Connected to: Stanford_Research_Systems SR830 (serial:s/n40974, firmware:ver1.07) in 0.26s


### Initialize the BundleLockin 
make a tuple of lockins 
and pass it to the BundleLockin.

Note:
Initializing the BundleLockin has the sideeffect
of setting buffer_SR = 'Trigger' and buffer_trig_mode = 'ON' for all the lockins in the bundle.

In [5]:
lockins = (sr,)
#lockins = (sr,sr2)
bundle = BundleLockin('bundle',lockins)



### do2d_multi
#### Args:
* param_slow: The QCoDeS parameter to sweep over in the outer loop
* start_slow: Starting point of sweep in outer loop
* stop_slow: End point of sweep in the outer loop
* num_points_slow: Number of points to measure in the outer loop
* delay_slow: Delay after setting parameter in the outer loop
* param_fast: The QCoDeS parameter to sweep over in the inner loop
* start_fast: Starting point of sweep in inner loop
* stop_fast: End point of sweep in the inner loop
* num_points_fast: Number of points to measure in the inner loop
* delay_fast: Delay after setting parameter before measurement is performed
* write_period: The time after which the data is actually written to the database.
* threading: For each ellement which are True, write_in_background, buffer_reset, and send_trigger and get_trace will be threaded respectively
* show_progress_bar: should a progress bar be displayed during the measurement.

In [6]:
do2d_multi(param_slow = slow.v_now,
           start_slow = 0,
           stop_slow = 0.5,
           num_points_slow = 10,
           delay_slow = 0.005,
           param_fast = fast.v_now,
           start_fast = 0,
           stop_fast = 0.5,
           num_points_fast = 10, 
           delay_fast = 0.045,
           bundle = bundle,
           threading=[True,False,False,True]
              )

c:\users\farzad\miniconda3\envs\sequencebuilder\lib\site-packages\qcodes\dataset\measurements.py:486: UserWarning: The specified write period of 1.0 s will be ignored, since write_in_background==True
  warnings.warn(f"The specified write period of {write_period} s "
  2% (2 of 100) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00

Starting experimental run with id: 3. 


 99% (99 of 100) |###################### | Elapsed Time: 0:00:06 ETA:   0:00:00